In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.simplefilter('ignore')

import gc

from os import path
import sys
sys.path.append(path.abspath('..'))

In [ ]:
import types
import numpy as np
from nvidia.dali.pipeline import Pipeline
import nvidia.dali.types as types
import nvidia.dali.ops as ops
from glob import glob

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
%matplotlib inline

def show_images(image_batch):
    columns = 4
    rows = (8 + 1) // (columns)
    fig = plt.figure(figsize = (32,(32 // columns) * rows))
    gs = gridspec.GridSpec(rows, columns)
    for j in range(rows * columns):
        plt.subplot(gs[j])
        plt.axis("off")
        plt.imshow(image_batch.at(j))

In [ ]:
class ExternalInputIterator:
    def __init__(self, batch_size: int):
        self.fnames = glob('/workspace/project/data/images/*.jpg')
        self.fnames.remove('/workspace/project/data/images/broken_image.jpg')
        self.batch_size = batch_size

    def __iter__(self):
        self.i = 0
        self.n = len(self.fnames)
        return self

    def __next__(self):
        batch = []
        labels = []
        for _ in range(self.batch_size):
            filename_jpg = self.fnames[self.i]
            label = np.random.randint(0, 2, size=(5,)).astype(np.float32)
            with open(filename_jpg, 'rb') as f:
                batch.append(np.frombuffer(f.read(), dtype=np.uint8))
            labels.append(label)
            self.i = (self.i + 1) % self.n
        return batch, labels

In [ ]:
class CustomPipeline(Pipeline):
    def __init__(self, batch_size: int, num_threads: int, device_id: int):
        super().__init__(batch_size=batch_size, num_threads=num_threads, device_id=device_id)

        self.input = ops.ExternalSource(source=ExternalInputIterator(batch_size), num_outputs=2)
        self.decoder = ops.ImageDecoder(device='mixed', output_type=types.RGB)
        self.resize = ops.Resize(device='gpu', size=(224, 224))

    def define_graph(self):
        jpegs, labels = self.input()
        images = self.decoder(jpegs)
        images = self.resize(images)

        return images, labels.gpu()

In [ ]:
pipeline = CustomPipeline(batch_size=16, num_threads=4, device_id=0)
pipeline.build()
output = pipeline.run()

In [ ]:
show_images(output[0].as_cpu())

In [ ]:
output = pipeline.run()
show_images(output[0].as_cpu())

In [ ]:
output = pipeline.run()
show_images(output[0].as_cpu())